In [0]:
ㄱfrom google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
%matplotlib inline
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import xgboost as xgb
import operator
from catboost import Pool, CatBoostClassifier
import lightgbm as lgb
import requests
from bs4 import BeautifulSoup
from datetime import datetime

In [0]:
# load data
df_train = pd.read_csv("/content/drive/My Drive/airbnb-recruiting-new-user-bookings/train_users_2.csv")
df_test = pd.read_csv("/content/drive/My Drive/airbnb-recruiting-new-user-bookings/test_users.csv")
#df_sessions = pd.read_csv("/content/drive/My Drive/airbnb-recruiting-new-user-bookings/sessions.csv")


## gender missing value imputation을 위한 list
Gend=['MALE','FEMALE']

In [0]:
age_list=list(df_train.age.unique())
age_list.sort()

### 참고: 각 column들의  unique한 value 값들과 개수

id :  ['gxn3p5htnn' '820tgsjxq7' '4ft3gnwmtx' ... '6o3arsjbb4' 'jh95kwisub'
 'nw9fwlyb5f'] , count:  213451

date_account_created :  ['2010-06-28' '2011-05-25' '2010-09-28' ... '2014-06-27' '2014-06-29'
 '2014-06-30'] , count:  1634
 
timestamp_first_active :  [20090319043255 20090523174809 20090609231247 ... 20140630235754
 20140630235822 20140630235824] , count:  213451
 
gender :  ['-unknown-' 'MALE' 'FEMALE' 'OTHER'] , count:  4

age : [nan,
 1.0,
 2.0,
 4.0,
 5.0,
 15.0,
 16.0,
 17.0,
 18.0,
 19.0,
 20.0,
 21.0,
 22.0,
 23.0,
 24.0,
 25.0,
 26.0,
 27.0,
 28.0,
 29.0,
 30.0,
 31.0,
 32.0,
 33.0,
 34.0,
 35.0,
 36.0,
 37.0,
 38.0,
 39.0,
 40.0,
 41.0,
 42.0,
 43.0,
 44.0,
 45.0,
 46.0,
 47.0,
 48.0,
 49.0,
 50.0,
 51.0,
 52.0,
 53.0,
 54.0,
 55.0,
 56.0,
 57.0,
 58.0,
 59.0,
 60.0,
 61.0,
 62.0,
 63.0,
 64.0,
 65.0,
 66.0,
 67.0,
 68.0,
 69.0,
 70.0,
 71.0,
 72.0,
 73.0,
 74.0,
 75.0,
 76.0,
 77.0,
 78.0,
 79.0,
 80.0,
 81.0,
 82.0,
 83.0,
 84.0,
 85.0,
 86.0,
 87.0,
 88.0,
 89.0,
 90.0,
 91.0,
 92.0,
 93.0,
 94.0,
 95.0,
 96.0,
 97.0,
 98.0,
 99.0,
 100.0,
 101.0,
 102.0,
 103.0,
 104.0,
 105.0,
 106.0,
 107.0,
 108.0,
 109.0,
 110.0,
 111.0,
 112.0,
 113.0,
 115.0,
 132.0,
 150.0,
 1924.0,
 1925.0,
 1926.0,
 1927.0,
 1928.0,
 1929.0,
 1931.0,
 1932.0,
 1933.0,
 1935.0,
 1936.0,
 1938.0,
 1942.0,
 1947.0,
 1949.0,
 1952.0,
 1953.0,
 1995.0,
 2008.0,
 2013.0,
 2014.0] , count:  128
 
signup_method :  ['facebook' 'basic' 'google'] , count:  3

signup_flow :  [ 0  3  2  1 24  8  6  5 10 25 12  4 16 15 20 21 23] , count:  17

language :  ['en' 'fr' 'de' 'es' 'it' 'pt' 'zh' 'ko' 'ja' 'ru' 'pl' 'el' 'sv' 'nl'
 'hu' 'da' 'id' 'fi' 'no' 'tr' 'th' 'cs' 'hr' 'ca' 'is'] , count:  25
 
affiliate_channel :  ['direct' 'seo' 'other' 'sem-non-brand' 'content' 'sem-brand'
 'remarketing' 'api'] , count:  8
 
affiliate_provider :  ['direct' 'google' 'other' 'craigslist' 'facebook' 'vast' 'bing' 'meetup'
 'facebook-open-graph' 'email-marketing' 'yahoo' 'padmapper' 'gsp' 'wayn'
 'naver' 'baidu' 'yandex' 'daum'] , count:  18
 
first_affiliate_tracked :  ['untracked' 'omg' nan 'linked' 'tracked-other' 'product' 'marketing'
 'local ops'] , count:  8
 
signup_app :  ['Web' 'Moweb' 'iOS' 'Android'] , count:  4

first_device_type :  ['Mac Desktop' 'Windows Desktop' 'iPhone' 'Other/Unknown'
 'Desktop (Other)' 'Android Tablet' 'iPad' 'Android Phone'
 'SmartPhone (Other)'] , count:  9
 
first_browser :  ['Chrome' 'IE' 'Firefox' 'Safari' '-unknown-' 'Mobile Safari'
 'Chrome Mobile' 'RockMelt' 'Chromium' 'Android Browser' 'AOL Explorer'
 'Palm Pre web browser' 'Mobile Firefox' 'Opera' 'TenFourFox' 'IE Mobile'
 'Apple Mail' 'Silk' 'Camino' 'Arora' 'BlackBerry Browser' 'SeaMonkey'
 'Iron' 'Sogou Explorer' 'IceWeasel' 'Opera Mini' 'SiteKiosk' 'Maxthon'
 'Kindle Browser' 'CoolNovo' 'Conkeror' 'wOSBrowser' 'Google Earth'
 'Crazy Browser' 'Mozilla' 'OmniWeb' 'PS Vita browser' 'NetNewsWire'
 'CometBird' 'Comodo Dragon' 'Flock' 'Pale Moon' 'Avant Browser'
 'Opera Mobile' 'Yandex.Browser' 'TheWorld Browser' 'SlimBrowser' 'Epic'
 'Stainless' 'Googlebot' 'Outlook 2007' 'IceDragon'] , count:  52

# 1. Preprocessing

In [0]:
## train set의 label 부분만 떼어옴. Series로 하지 않고 .values를 통해 array type으로 전환
labels = df_train['country_destination'].values # Get the values of the country destination for each row

## 앞에서 labels에 label 정보를 저장했으므로, df_train에서는 label 정보를 뺀 피쳐들만 남김 
#df_train = df_train.drop(['country_destination'], axis=1) # It's the output variable for the decision tree <<?? input variable의 오타?

## test set의 id만 떼어옴. Series 타입
id_test = df_test['id']

## train set의 instance 수
piv_train = df_train.shape[0] # Size of df_train (number of rows)

In [0]:
## train set에 'account_created'이라는 이름의 feature 추가. 이 feature은 train set의 'date_account_created'을 날짜-시간 type으로 변경한 것(string type->datetime64[ns] type)
## datetime의 형식 '2010-06-28T00:00:00.000000000'
df_train['account_created']=pd.to_datetime(df_train.date_account_created)

## test set에 'account_created'이라는 이름의 feature 추가. 이 feature은 test set의 'date_account_created'을 날짜-시간 type으로 변경한 것
df_test['account_created']=pd.to_datetime(df_test.date_account_created)

In [0]:
## train set의 'age' feature을 array 형식으로 저장한다
av = df_train.age.values   #1914<age<2004-->date account created , age<10,110<age<151,age>2003-->mean age

## test set의 'age' feature을 array 형식으로 저장한다
av2=df_test.age.values

## 'age' artifact 중 1914<age<2004 인 경우 처리. 조건이 true일 시 2번 째 argument를, false일 시 3번 째 argument를 값으로 취한다.
## 'age' 값이 연도 같은 형식을 취한 경우, 나이가 아닌 출생년도로 간주하여 처리하는 것.
## 1914<age<2004 인 경우: 2번 째 argument=(account_created 연도) - (기존에 있던 값)
## 해당하지 않는 경우 기존 값 그대로 둠
df_train['age'] = np.where(np.logical_and(av>1914, av<2004),df_train.account_created.dt.year-av, av)
df_test['age'] = np.where(np.logical_and(av2>1914, av2<2004),df_test.account_created.dt.year-av2, av2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in greater
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in less
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in greater
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in less
  # This is added back by InteractiveShellApp.init_path()


In [0]:
## 'gender'의 경우, 값이 '-unknown-'이면 'Male', 'Female' 중 랜덤하게 넣는다.
df_train['gender']=df_train['gender'].map(lambda x:random.choice(Gend) if x== '-unknown-' else x)
df_test['gender']=df_test['gender'].map(lambda x:random.choice(Gend) if x== '-unknown-' else x)

In [0]:
## train set과 test set이 하나로 합쳐진 dataframe 생성. 같은 transformation을 둘에 각각 할 필요없이 공통으로 한 번에 처리하기 위함.
# We will separate both of them when applying the decision tree classification
df_all = pd.concat((df_train, df_test), axis = 0, ignore_index = True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [0]:
# We remove the id and the date of first booking since it is not used as an input to our classifier
df_all = df_all.drop(['date_first_booking'], axis=1)
## 위에 주석처리 된 것이 기존에 받은 상태였고, id는 sessions data와 join해야 하므로 마지막에 모델에 돌리기  삭제

In [0]:
df_train.head(10)

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination,account_created
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,MALE,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF,2010-06-28
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF,2011-05-25
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US,2010-09-28
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other,2011-12-05
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,MALE,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US,2010-09-14
5,osr2jwljor,2010-01-01,20100101215619,2010-01-02,MALE,NaN,basic,0,en,other,other,omg,Web,Mac Desktop,Chrome,US,2010-01-01
6,lsw9q7uk0j,2010-01-02,20100102012558,2010-01-05,FEMALE,46.0,basic,0,en,other,craigslist,untracked,Web,Mac Desktop,Safari,US,2010-01-02
7,0d01nltbrs,2010-01-03,20100103191905,2010-01-13,FEMALE,47.0,basic,0,en,direct,direct,omg,Web,Mac Desktop,Safari,US,2010-01-03
8,a1vcnhxeij,2010-01-04,20100104004211,2010-07-29,FEMALE,50.0,basic,0,en,other,craigslist,untracked,Web,Mac Desktop,Safari,US,2010-01-04
9,6uh8zyj2gn,2010-01-04,20100104023758,2010-01-04,MALE,46.0,basic,0,en,other,craigslist,omg,Web,Mac Desktop,Firefox,US,2010-01-04


In [0]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275547 entries, 0 to 275546
Data columns (total 16 columns):
account_created            275547 non-null datetime64[ns]
affiliate_channel          275547 non-null object
affiliate_provider         275547 non-null object
age                        158681 non-null float64
country_destination        213451 non-null object
date_account_created       275547 non-null object
first_affiliate_tracked    269462 non-null object
first_browser              275547 non-null object
first_device_type          275547 non-null object
gender                     275547 non-null object
id                         275547 non-null object
language                   275547 non-null object
signup_app                 275547 non-null object
signup_flow                275547 non-null int64
signup_method              275547 non-null object
timestamp_first_active     275547 non-null int64
dtypes: datetime64[ns](1), float64(1), int64(2), object(12)
memory usage: 33.6+ MB

### missing value 처리

In [0]:
## 'date_account_created'의 년, 월, 일을 각각 서로 다른 3개의 feature로 추가한다.
# Date_account_created transformation
# Separate the day month and year into different columns
dac = np.vstack(df_all.date_account_created.astype(str).apply(lambda x: list(map(int, x.split('-')))).values)
print(dac)
df_all['dac_year'] = dac[:,0]
df_all['dac_month'] = dac[:,1]
df_all['dac_day'] = dac[:,2]

## 기존에 받았을 때는 'account_created'만 drop하는 걸로 되어있었는데, 'date_account_created'까지 drop하는 것으로 수정함.->account_created는 날짜 처리 이후 drop으로 처리 
df_all = df_all.drop(['date_account_created'], axis = 1)

[[2010    6   28]
 [2011    5   25]
 [2010    9   28]
 ...
 [2014    9   30]
 [2014    9   30]
 [2014    9   30]]


In [0]:
## 'timestamp_first_active'의 년, 월, 일을 각각 서로 다른 3개의 feature로 추가한다.
# timestamp_first_active transformation
# Here as well we separate the timestamp first active into day month and year
tfa = np.vstack(df_all.timestamp_first_active.astype(str).apply(lambda x: list(map(int, [x[:4],x[4:6],x[6:8],x[8:10],x[10:12],x[12:14]]))).values)
df_all['tfa_year'] = tfa[:,0]
df_all['tfa_month'] = tfa[:,1]
df_all['tfa_day'] = tfa[:,2]

## 기존에 받았을 때에는 시간 정보는 추가하지 않았는데 시간 정보까지 있으면 좋을 것 같아 추가함.
df_all['tfa_time'] = tfa[:,3]

In [0]:
## 기존에 받았을 때에는 drop이 주석처리되어있었는데 주석 해제하여 'timestamp_first_active' drop함.
df_all = df_all.drop(['timestamp_first_active'], axis=1)

In [0]:
## df_all, df_train, df_test의 row 개수 확인
print(df_all.shape)
print(df_train.shape)
print(df_test.shape)

(275547, 21)
(213451, 17)
(62096, 16)


In [0]:
## 해당 범위의 age값들을 모두 nan으로 통일한 후 mean을 계산하여, nan인 value들에 일괄적으로 mean을 채워넣는다.
## 우선, 이상한 값들을 모두 nan으로 전환한다.
av = df_all.age.values
df_all['age'] = np.where(av<10, np.nan, av)
df_all['age'] = np.where(av>2003, np.nan, av)
df_all['age'] = np.where(av>110, np.nan, av)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in less
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in greater
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in greater
  after removing the cwd from sys.path.


In [0]:
## 기존에 받았을 때는 train, test set의 평균을 채웠었는데, 원칙적으로 missing value imputation의 기준은 train set에 기반해야 하며, test set의 정보는 제외시켜야 한다.
## 따라서 train, test 합친 set이 아닌 train set의 'age' 평균값을 받는 것으로 수정함.
# Age transformation
av = df_all.age.values
print(df_all[:213451].age.describe())
mean_age=int(df_all[:213451].age.mean())

count    124632.000000
mean         37.423631
std          13.917456
min          15.000000
25%          28.000000
50%          34.000000
75%          43.000000
max         110.000000
Name: age, dtype: float64


In [0]:
## 일괄적으로 missing age impute
df_all.age=df_all.age.fillna(value=mean_age)
df_all.head()

,account_created,affiliate_channel,affiliate_provider,age,country_destination,first_affiliate_tracked,first_browser,first_device_type,gender,id,language,signup_app,signup_flow,signup_method,dac_year,dac_month,dac_day,tfa_year,tfa_month,tfa_day,tfa_time
0,2010-06-28,direct,direct,37.0,NDF,untracked,Chrome,Mac Desktop,MALE,gxn3p5htnn,en,Web,0,facebook,2010,6,28,2009,3,19,4
1,2011-05-25,seo,google,38.0,NDF,untracked,Chrome,Mac Desktop,MALE,820tgsjxq7,en,Web,0,facebook,2011,5,25,2009,5,23,17
2,2010-09-28,direct,direct,56.0,US,untracked,IE,Windows Desktop,FEMALE,4ft3gnwmtx,en,Web,3,basic,2010,9,28,2009,6,9,23
3,2011-12-05,direct,direct,42.0,other,untracked,Firefox,Mac Desktop,FEMALE,bjjt8pjhuk,en,Web,0,facebook,2011,12,5,2009,10,31,6
4,2010-09-14,direct,direct,41.0,US,untracked,Chrome,Mac Desktop,MALE,87mebub9p4,en,Web,0,basic,2010,9,14,2009,12,8,6


### 추가적 Feature 생성

In [0]:
# 1. dac_pred_diff=(year_of_prediction) - (dac_year): 계정생성한지 얼마나 되었나(연도 기준)
df_all['dac_pred_diff']=2015-df_all['dac_year']

In [0]:
# 2. dac_month 계절 (춘하추동 4가지)
# -춘: 3, 4, 5 -하: 6, 7, 8 -추: 9, 10, 11 -동: 12, 1, 2
df_all['dac_season']=df_all['dac_month'].map(lambda x:'spring' if x in [3, 4, 5] else('summer' if x in [6, 7, 8] else('fall' if x in [9, 10, 11] else 'winter')))

In [0]:
import datetime

In [0]:
# 3. dac 요일
df_all['dac_weekday']= df_all['account_created'].dt.day_name()

df_all = df_all.drop(['account_created'], axis = 1)

In [0]:
# 1. tfa_pred_diff=(year_of_prediction) - (tfa_year): 첫 활동한지 얼마나 되었나(연도 기준)
df_all['tfa_pred_diff']=2015-df_all['tfa_year']

In [0]:
# 2. tfa_month 계절 (춘하추동 4가지)
# -춘: 3, 4, 5 -하: 6, 7, 8 -추: 9, 10, 11 -동: 12, 1, 2
df_all['tfa_season']=df_all['tfa_month'].map(lambda x:'spring' if x in [3, 4, 5] else('summer' if x in [6, 7, 8] else('fall' if x in [9, 10, 11] else 'winter')))

In [0]:
# 3. tfa의 요일
df_all['tfa']=df_all['tfa_year'].map(lambda x: str(x))+ '-' +df_all['tfa_month'].map(lambda x: str(x) if x in [10, 11, 12] else '0'+str(x))+ '-' +df_all['tfa_day'].map(lambda x: '0'+str(x) if x in [1, 2, 3, 4, 5, 6, 7, 8, 9] else str(x))
df_all['tfa']=pd.to_datetime(df_all['tfa'])
df_all['tfa_weekday']= df_all['tfa'].dt.day_name()

df_all = df_all.drop(['tfa'], axis = 1)

In [0]:
# time은 미국 내 시차 때문에 빼는 게 나을 수도 있다 ->timestamp 찍힐 때의 기준?
# 4. tfa_time이 어느 시간대인지 (밤, 저녁, 낮, 아침, 새벽)
# -밤: 21~3  
# -저녁: 18~20
# -낮: 12~17
# -아침: 7~11
# -새벽: 4~6
df_all['tfa_trange']=df_all['tfa_time'].map(lambda x:'night' if x in [21, 22, 23, 24, 1, 2, 3] else('evening' if x in [18, 19, 20] else('afternoon' if x in [12, 13, 14, 15, 16, 17] else('dawn' if x in [4, 5, 6] else 'morning'))))

In [0]:
df_all.head()

,affiliate_channel,affiliate_provider,age,country_destination,first_affiliate_tracked,first_browser,first_device_type,gender,id,language,signup_app,signup_flow,signup_method,dac_year,dac_month,dac_day,tfa_year,tfa_month,tfa_day,tfa_time,dac_pred_diff,dac_season,dac_weekday,tfa_pred_diff,tfa_weekday,tfa_trange
0,direct,direct,37.0,NDF,untracked,Chrome,Mac Desktop,MALE,gxn3p5htnn,en,Web,0,facebook,2010,6,28,2009,3,19,4,5,summer,Monday,6,Thursday,dawn
1,seo,google,38.0,NDF,untracked,Chrome,Mac Desktop,MALE,820tgsjxq7,en,Web,0,facebook,2011,5,25,2009,5,23,17,4,spring,Wednesday,6,Saturday,afternoon
2,direct,direct,56.0,US,untracked,IE,Windows Desktop,FEMALE,4ft3gnwmtx,en,Web,3,basic,2010,9,28,2009,6,9,23,5,fall,Tuesday,6,Tuesday,night
3,direct,direct,42.0,other,untracked,Firefox,Mac Desktop,FEMALE,bjjt8pjhuk,en,Web,0,facebook,2011,12,5,2009,10,31,6,4,winter,Monday,6,Saturday,dawn
4,direct,direct,41.0,US,untracked,Chrome,Mac Desktop,MALE,87mebub9p4,en,Web,0,basic,2010,9,14,2009,12,8,6,5,fall,Tuesday,6,Tuesday,dawn


In [0]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275547 entries, 0 to 275546
Data columns (total 26 columns):
affiliate_channel          275547 non-null object
affiliate_provider         275547 non-null object
age                        275547 non-null float64
country_destination        213451 non-null object
first_affiliate_tracked    269462 non-null object
first_browser              275547 non-null object
first_device_type          275547 non-null object
gender                     275547 non-null object
id                         275547 non-null object
language                   275547 non-null object
signup_app                 275547 non-null object
signup_flow                275547 non-null int64
signup_method              275547 non-null object
dac_year                   275547 non-null int64
dac_month                  275547 non-null int64
dac_day                    275547 non-null int64
tfa_year                   275547 non-null int64
tfa_month                  275547 non-null 

In [0]:
# 일단 user 데이터만 중간저장
# train(213451, )  test(62096, )
new_train=df_all[:213451]
new_test=df_all[213451:]

In [0]:
new_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213451 entries, 0 to 213450
Data columns (total 26 columns):
affiliate_channel          213451 non-null object
affiliate_provider         213451 non-null object
age                        213451 non-null float64
country_destination        213451 non-null object
first_affiliate_tracked    207386 non-null object
first_browser              213451 non-null object
first_device_type          213451 non-null object
gender                     213451 non-null object
id                         213451 non-null object
language                   213451 non-null object
signup_app                 213451 non-null object
signup_flow                213451 non-null int64
signup_method              213451 non-null object
dac_year                   213451 non-null int64
dac_month                  213451 non-null int64
dac_day                    213451 non-null int64
tfa_year                   213451 non-null int64
tfa_month                  213451 non-null 

In [0]:
new_train.to_csv("/content/drive/My Drive/airbnb-recruiting-new-user-bookings/procesed_train_users.csv", index=False)
new_test.to_csv("/content/drive/My Drive/airbnb-recruiting-new-user-bookings/processed_test_users.csv", index=False)

## model에 넣기 전에 처리

In [0]:
# Z-score nomalization: age, dac_pred_diff, tfa_pred_diff ->float64, int64 type
# One-hot encoding: gender, signup_flow<-해석의 가치는 없어서 모델 때부터 뺄지 고려, language, etc. ->object type이면 전부 + dac_year과 tfa_year
# 날짜, 시간과 같은 ordinal한 data도 Z-score, 단 dac_pred_diff, tfa_pred_diff가 시간차를 담는 기능을 담당하므로 dac_year과 tfa_year 같은 연도 data는 이와 구별되게 '특정 해'를 나타내는 Categorical로 취급
# id 등 안되는 feature 제거
